In [ ]:
# Price & Trades Analysis
# Interactive visualization of OHLC data with trade entries and exits
# Ported from dash app to notebook for easier analysis
#
# Chart interactivity:
# - Scroll wheel: zoom in/out
# - Drag: pan around
# - Double-click: reset zoom
# - Hover: see trade PnL details

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly_resampler import FigureResampler, FigureWidgetResampler
from plotly_resampler.aggregation import MinMaxLTTB
import os
import sys

# Add research directory to path
sys.path.append('/home/stan/src/download-polygon-aggregates/research')
from data_downloader import get_filename

In [ ]:
# Load OHLC bars and trades data
# Configuration
ticker = 'AAPL'
interval = '5s'
year = '2024'

# Load OHLC data
pwd = '/home/stan/src/download-polygon-aggregates/research/dash'
filename = get_filename(ticker, interval, year)
bars = pd.read_parquet(f'{pwd}/../{filename}')
bars['timestamp'] = pd.to_datetime(bars['timestamp'], unit='s')

# Load trades data
trades = pd.read_csv(f"{pwd}/trades.csv", parse_dates=["entry_ts", "exit_ts"]).sort_values("entry_ts")

print(f"Loaded {len(bars):,} bars and {len(trades):,} trades")
print(f"Data period: {bars['timestamp'].min()} to {bars['timestamp'].max()}")
print(f"Trades period: {trades['entry_ts'].min()} to {trades['exit_ts'].max()}")

In [ ]:
# Helper functions for filtering trades and creating charts

def filter_trades(trades_df, params_hash=None, stop_loss=None, side="ALL"):
    """Filter trades by parameters"""
    df = trades_df.copy()
    
    if params_hash is not None:
        df = df[df["params_hash"].astype(str) == str(params_hash)]
    
    if stop_loss is not None:
        df = df[df["stop_loss"] == stop_loss]
    
    if side == "LONG":
        df = df[df["side"].str.lower() == "long"]
    elif side == "SHORT":
        df = df[df["side"].str.lower() == "short"]
    
    return df

def create_price_chart(bars_df, trades_df, mode="line", title="Price & Trades"):
    """Create price chart with trades overlay"""
    timestamps_np = bars_df["timestamp"].values
    
    if mode == "candles":
        fig = go.Figure()
        fig.add_trace(go.Candlestick(
            x=timestamps_np, 
            open=bars_df["open"], 
            high=bars_df["high"],
            low=bars_df["low"], 
            close=bars_df["close"], 
            name="Price", 
            showlegend=False
        ))
    else:
        # Line mode with FigureWidgetResampler for proper Jupyter integration
        fig = FigureWidgetResampler(
            default_n_shown_samples=1000,
            default_downsampler=MinMaxLTTB(parallel=False)
        )
        fig.add_trace(
            go.Scattergl(mode="lines", name="Close", line=dict(width=1)),
            hf_x=timestamps_np, hf_y=bars_df["close"]
        )
        fig.add_trace(
            go.Scattergl(mode="lines", name="High", line=dict(width=1, color="gray")),
            hf_x=timestamps_np, hf_y=bars_df["high"]
        )
        fig.add_trace(
            go.Scattergl(mode="lines", name="Low", line=dict(width=1, color="gray")),
            hf_x=timestamps_np, hf_y=bars_df["low"]
        )
    
    # Add trade markers
    if not trades_df.empty:
        # Entry points
        fig.add_trace(go.Scattergl(
            x=trades_df["entry_ts"].values, 
            y=trades_df["entry_price"], 
            mode="markers", 
            name="Entry",
            marker=dict(symbol="triangle-up", size=8, color="blue")
        ))
        
        # Exit points (color by PnL)
        exit_colors = np.where(trades_df["pnl"] >= 0, "green", "red")
        fig.add_trace(go.Scattergl(
            x=trades_df["exit_ts"].values, 
            y=trades_df["exit_price"], 
            mode="markers", 
            name="Exit",
            marker=dict(symbol="x", size=8, color=exit_colors),
            text=[f"PnL: {pnl:.2f}" for pnl in trades_df["pnl"]],
            hovertemplate="%{text}<extra></extra>"
        ))
    
    fig.update_layout(
        title=title,
        height=600,
        xaxis_rangeslider_visible=False,
        showlegend=True,
        # Enable better interactivity
        dragmode='zoom',  # Default to zoom mode
        hovermode='x unified'  # Better hover info
    )
    
    return fig

# Get available filter options
params_options = sorted(trades["params_hash"].astype(str).unique().tolist())
stoploss_options = sorted(trades["stop_loss"].unique().tolist())
side_options = ["ALL", "LONG", "SHORT"]

print(f"Available params_hash: {params_options[:5]}... ({len(params_options)} total)")
print(f"Available stop_loss: {stoploss_options}")
print(f"Available sides: {side_options}")

In [ ]:
# Quick analysis with default parameters
# Use first available parameters for quick view
default_params = params_options[0] if params_options else None
default_stoploss = stoploss_options[0] if stoploss_options else None

filtered_trades = filter_trades(trades, 
                               params_hash=default_params, 
                               stop_loss=default_stoploss, 
                               side="ALL")

print(f"Filtered to {len(filtered_trades)} trades")
print(f"Parameters: params_hash={default_params}, stop_loss={default_stoploss}")

# Show sample of trades
display(filtered_trades.head())

In [ ]:
# Note about FigureWidgetResampler:
# - Automatically resamples data when you zoom in/out
# - Works only in "line" mode (not "candles")
# - Shows ~1000 samples by default, adds more detail when zooming
# - Use scroll wheel or selection zoom - resampling happens automatically
print("FigureWidgetResampler ready - zoom in/out to see automatic resampling!")

In [ ]:
# Price chart with line mode (good for large datasets)
# FigureWidgetResampler automatically handles zoom resampling in Jupyter
fig_line = create_price_chart(bars, filtered_trades, mode="line", 
                             title=f"Price & Trades (Line) - {ticker} {interval}")
fig_line  # Display directly for widget integration

In [ ]:
# Custom analysis - modify these parameters as needed
PARAMS_HASH = params_options[0]  # Change index or set specific value
STOP_LOSS = stoploss_options[0]   # Change index or set specific value  
SIDE = "ALL"                      # "ALL", "LONG", or "SHORT"
CHART_MODE = "line"               # "line" or "candles"

# Filter trades
custom_trades = filter_trades(trades, 
                             params_hash=PARAMS_HASH, 
                             stop_loss=STOP_LOSS, 
                             side=SIDE)

print(f"Filtered to {len(custom_trades)} trades")
print(f"Total PnL: {custom_trades['pnl'].sum():.2f}")
print(f"Win rate: {(custom_trades['pnl'] > 0).mean():.1%}")
print(f"Avg PnL: {custom_trades['pnl'].mean():.2f}")

In [ ]:
# Custom chart with selected parameters
fig_custom = create_price_chart(bars, custom_trades, mode=CHART_MODE,
                               title=f"Custom Analysis - {SIDE} trades, SL={STOP_LOSS}")
fig_custom  # Display directly for widget integration

In [ ]:
# Trades table with key columns
# Display trades table (can be sorted by clicking column headers in some notebook environments)
trades_display = custom_trades[[
    'trade_id', 'side', 'entry_ts', 'entry_price', 
    'exit_ts', 'exit_price', 'pnl', 'pnl_pct', 
    'stop_loss', 'duration_s'
]].round(5)

print(f"Showing {len(trades_display)} trades:")
display(trades_display)

In [ ]:
# Quick statistics summary
if not custom_trades.empty:
    stats = {
        'Total Trades': len(custom_trades),
        'Total PnL': custom_trades['pnl'].sum(),
        'Win Rate': (custom_trades['pnl'] > 0).mean(),
        'Average PnL': custom_trades['pnl'].mean(),
        'Best Trade': custom_trades['pnl'].max(),
        'Worst Trade': custom_trades['pnl'].min(),
        'Avg Duration (s)': custom_trades['duration_s'].mean(),
        'Long Trades': (custom_trades['side'].str.lower() == 'long').sum(),
        'Short Trades': (custom_trades['side'].str.lower() == 'short').sum(),
    }
    
    stats_df = pd.DataFrame(list(stats.items()), columns=['Metric', 'Value'])
    display(stats_df)
else:
    print("No trades found for selected filters")

In [ ]:
# Alternative function for candles with large datasets
def create_price_chart_subsampled(bars_df, trades_df, subsample_factor=10, title="Price & Trades (Subsampled)"):
    """Create candlestick chart with subsampled data for performance"""
    # Subsample bars for performance
    bars_sub = bars_df.iloc[::subsample_factor].copy()
    
    fig = go.Figure()
    fig.add_trace(go.Candlestick(
        x=bars_sub["timestamp"], 
        open=bars_sub["open"], 
        high=bars_sub["high"],
        low=bars_sub["low"], 
        close=bars_sub["close"], 
        name="Price", 
        showlegend=False
    ))
    
    # Add trade markers (don't subsample trades)
    if not trades_df.empty:
        # Entry points
        fig.add_trace(go.Scattergl(
            x=trades_df["entry_ts"], 
            y=trades_df["entry_price"], 
            mode="markers", 
            name="Entry",
            marker=dict(symbol="triangle-up", size=8, color="blue")
        ))
        
        # Exit points
        exit_colors = np.where(trades_df["pnl"] >= 0, "green", "red")
        fig.add_trace(go.Scattergl(
            x=trades_df["exit_ts"], 
            y=trades_df["exit_price"], 
            mode="markers", 
            name="Exit",
            marker=dict(symbol="x", size=8, color=exit_colors),
            text=[f"PnL: {pnl:.2f}" for pnl in trades_df["pnl"]],
            hovertemplate="%{text}<extra></extra>"
        ))
    
    fig.update_layout(
        title=title,
        height=600,
        xaxis_rangeslider_visible=False,
        showlegend=True,
        dragmode='zoom',
        hovermode='x unified'
    )
    
    return fig

# Example: create subsampled candles chart (every 10th bar)
if len(bars) > 50000:  # Only if dataset is large
    print(f"Large dataset detected ({len(bars):,} bars). Creating subsampled candles chart...")
    fig_candles_sub = create_price_chart_subsampled(bars, custom_trades, subsample_factor=20,
                                                   title=f"Candles (1/{20} subsampled) - {ticker} {interval}")
    fig_candles_sub.show(config={'scrollZoom': True})